### source code from TFA

In [ ]:
# if cfg.MODEL.BACKBONE.FREEZE:
#     for p in self.backbone.parameters():
#         p.requires_grad = False
#     print("froze backbone parameters")

# if cfg.MODEL.PROPOSAL_GENERATOR.FREEZE:
#     for p in self.proposal_generator.parameters():
#         p.requires_grad = False
#     print("froze proposal generator parameters")

# if cfg.MODEL.ROI_HEADS.FREEZE_FEAT:
#     for p in self.roi_heads.box_head.parameters():
#         p.requires_grad = False
#     print("froze roi_box_head parameters")

### check gradient visualization

### visualize NEU_DET

In [4]:
!python /home/dlsuncheng/FSOD/FsMMdet/MMdet/tools/misc/browse_dataset.py /home/dlsuncheng/FSOD/FsMMdet/Model/full_data/frcn_all_zero_dce.py \
    --output-dir /home/dlsuncheng/Dataset/NEU-DET/NEU-DET-Zero-DCE-Visual/ 

/home/dlsuncheng/FSOD/FsMMdet/MMdet/tools/misc/browse_dataset.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
[                                                  ] 0/1800, elapsed: 0s, ETA:Figure(800.01x800.01)
[                                ] 1/1800, 1.7 task/s, elapsed: 1s, ETA:  1065sFigure(800.01x800.01)
[                                ] 2/1800, 1.5 task/s, elapsed: 1s, ETA:  1206sFigure(800.01x800.01)
[                                ] 3/1800, 1.5 task/s, elapsed: 2s, ETA:  1231sFigure(800.01x800.01)
[                                ] 4/1800, 1.4 task/s, elapsed: 3s, ETA:  1278sFigure(800.01x800.01)
[                                ] 5/1800, 1.3 task/s, elapsed: 4s, ETA:  1426sFigure(800.01x800.01)
[                                ] 6/1800, 1.

### coco post analysis

#### 1. 总体精度        
#### 2. 是否存在错误分类  
#### 3. 是否存在漏检
#### 4. 结果可视化

In [1]:
# pth_path = "/home/dlsuncheng/Work_dir/Steel_Defect/20211021/FRCN_all/epoch_21.pth"
import json
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
def Read_Json(path):
    with open(path,"r") as load_f:
        json_file = json.load(load_f)
    return json_file

result_path = "./Result/FRCN_all_test.bbox.json"
test_anno_path = "/home/dlsuncheng/FSOD/FsMMdet/Datasets/NEU_DET/COCO_Annotation/test.json"
result_json = Read_Json(result_path)
test_json = Read_Json(test_anno_path)

img_info = pd.DataFrame(columns=["image_id","gt_category","pred_category"])
gt_image_id_list = [img_info["image_id"] for img_info in test_json["annotations"]]
gt_image_id = list(set(gt_image_id_list))
gt_image_id.sort()
img_info["image_id"] = gt_image_id

## 误分类分析
### 同一张图片内的类别应该相同
### image_id:[category_ids]
pred_category = []
gt_category = []
for img_id in gt_image_id:
    pred_cat_id_list = np.array([img["category_id"] for img in result_json if img["image_id"]==img_id])
    gt_cat_id_list = np.array([anno["category_id"] for anno in test_json["annotations"] if anno["image_id"]==img_id])
    gt_cat_id = gt_cat_id_list[0]*np.ones(pred_cat_id_list.shape)
    pred_category.extend(pred_cat_id_list)
    gt_category.extend(gt_cat_id)

con=confusion_matrix(gt_category,pred_category)
# sn.set()

# plt.rcParams['font.sans-serif']=['SimHei']
# fig = plt.figure(figsize=(10,10))
# sn.heatmap(con,annot = True,linewidths=1)
# plt.savefig("./cls_heat.png")
# plt.show()
### 是否存在漏检
# pred_image_id_list = [img_info["image_id"] for img_info in result_json]
# pred_bbox_num = dict(Counter(pred_image_id_list))
# gt_image_id_list = [img_info["image_id"] for img_info in test_json["annotations"]]
# gt_bbox_num = dict(Counter(gt_image_id_list))

# pred_num = list(pred_bbox_num.values())
# gt_num = list(gt_bbox_num.values())
# for index in range(len(pred_num)):
#     if pred_num[index]-gt_num[index]<0:
#         print(1)

## 没有漏检，但是部分图片多出来了很多bbox

### 权重——去除最后一个分类层，和其他多余信息

In [4]:
import torch
pth_path = "/home/dlsuncheng/Work_dir/Steel_Defect/20211021/FRCN_cos-ft/epoch_24.pth"
save_path = "/home/dlsuncheng/FSOD/FsMMdet/Weights/weights_removed.pth"
def reset_ckpt(ckpt):
    if 'meta' in ckpt:
        del ckpt['meta']
    if 'optimizer' in ckpt:
        del ckpt['optimizer']
    if 'iteration' in ckpt:
        ckpt['iteration'] = 0

def save_ckpt(ckpt, save_name):
    torch.save(ckpt, save_name)
    print('save changed ckpt to {}'.format(save_name))

ckpt = torch.load(pth_path)

param_name = 'roi_head.bbox_head'

for parameter in list(ckpt["state_dict"].keys()):
    if "fc_cls" in parameter or "fc_reg" in parameter:
        del ckpt['state_dict'][parameter]
reset_ckpt(ckpt)
save_ckpt(ckpt,save_path)

save changed ckpt to /home/dlsuncheng/FSOD/FsMMdet/Weights/weights_removed.pth


In [1]:
import torch
tfa_cos_path = "/home/dlsuncheng/Work_dir/Steel_Defect/20211021/FRCN_all/epoch_15.pth"
# tfa_fc_path = "/home/dlsuncheng/FSOD/FsMMdet/Weights/tfa_fc.pth"

tfa_cos_pth = torch.load(tfa_cos_path)
# tfa_fc_pth = torch.load(tfa_fc_path)
for param_name in tfa_cos_pth["state_dict"].keys():
    # if "roi_head" in param_name:
    print(param_name)
    print(tfa_cos_pth["state_dict"][param_name].shape)


# print([param_name for param_name in tfa_cos_pth["model"].keys() if "roi_heads" in param_name])
# print([param_name for param_name in tfa_fc_pth["model"].keys() if "roi_heads" in param_name])

backbone.conv1.weight
torch.Size([64, 3, 7, 7])
backbone.bn1.weight
torch.Size([64])
backbone.bn1.bias
torch.Size([64])
backbone.bn1.running_mean
torch.Size([64])
backbone.bn1.running_var
torch.Size([64])
backbone.bn1.num_batches_tracked
torch.Size([])
backbone.layer1.0.conv1.weight
torch.Size([64, 64, 1, 1])
backbone.layer1.0.bn1.weight
torch.Size([64])
backbone.layer1.0.bn1.bias
torch.Size([64])
backbone.layer1.0.bn1.running_mean
torch.Size([64])
backbone.layer1.0.bn1.running_var
torch.Size([64])
backbone.layer1.0.bn1.num_batches_tracked
torch.Size([])
backbone.layer1.0.conv2.weight
torch.Size([64, 64, 3, 3])
backbone.layer1.0.bn2.weight
torch.Size([64])
backbone.layer1.0.bn2.bias
torch.Size([64])
backbone.layer1.0.bn2.running_mean
torch.Size([64])
backbone.layer1.0.bn2.running_var
torch.Size([64])
backbone.layer1.0.bn2.num_batches_tracked
torch.Size([])
backbone.layer1.0.conv3.weight
torch.Size([256, 64, 1, 1])
backbone.layer1.0.bn3.weight
torch.Size([256])
backbone.layer1.0.bn3.bia

In [5]:
import json
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
def Read_Json(path):
    with open(path,"r") as load_f:
        json_file = json.load(load_f)
    return json_file
json_path = "/home/dlsuncheng/FSOD/FsMMdet/Datasets/NEU_DET/COCO_Annotation/instances_all.json"
json_file = Read_Json(json_path)
for anno in json_file["annotations"]:
    print(anno["bbox"])
    print(anno["bbox"][0]+anno["bbox"][2])
    print(anno["bbox"][1]+anno["bbox"][3])

        # print(anno)

[0, 53, 62, 37]
62
90
[93, 0, 77, 75]
170
75
[73, 108, 111, 91]
184
199
[143, 92, 32, 27]
175
119
[155, 46, 43, 29]
198
75
[78, 54, 47, 145]
125
199
[33, 3, 41, 83]
74
86
[100, 16, 79, 154]
179
170
[24, 158, 31, 41]
55
199
[41, 0, 67, 78]
108
78
[115, 46, 68, 62]
183
108
[49, 115, 48, 85]
97
200
[86, 0, 21, 47]
107
47
[41, 78, 66, 120]
107
198
[0, 15, 170, 93]
170
108
[0, 119, 168, 70]
168
189
[70, 40, 55, 137]
125
177
[144, 38, 49, 44]
193
82
[3, 0, 57, 172]
60
172
[157, 94, 42, 48]
199
142
[99, 127, 64, 43]
163
170
[29, 32, 66, 66]
95
98
[5, 17, 166, 179]
171
196
[52, 7, 125, 82]
177
89
[0, 0, 200, 199]
200
199
[21, 108, 80, 88]
101
196
[0, 36, 68, 164]
68
200
[99, 106, 57, 92]
156
198
[100, 30, 66, 79]
166
109
[0, 121, 93, 12]
93
133
[0, 148, 197, 17]
197
165
[10, 32, 62, 49]
72
81
[120, 31, 51, 85]
171
116
[153, 152, 36, 47]
189
199
[55, 53, 39, 114]
94
167
[147, 164, 29, 35]
176
199
[166, 39, 13, 82]
179
121
[94, 0, 106, 198]
200
198
[1, 0, 98, 198]
99
198
[111, 45, 88, 93]
199
13

In [ ]:
from albumentations import Equalize,CLAHE,RandomGamma,RandomContrast
import matplotlib.pyplot as plt
import cv2 as cv
# from mmdet.datasets.pipelines.transforms import Retinex
# img = cv2.imread(r"/home/dlsuncheng/FSOD/FsMMdet/Datasets/NEU_DET/Zero-DCE-NEU-DET/crazing_6.jpg")
img = cv.imread(r"/home/dlsuncheng/FSOD/FsMMdet/Datasets/NEU_DET/Zero-DCE-NEU-DET/crazing_20.jpg")
img_thr = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,5))
tophat = cv.morphologyEx(img_thr,cv.MORPH_TOPHAT,kernel,iterations=1)


clahe = CLAHE(clip_limit=(1, 4), tile_grid_size=(8, 8), always_apply=False, p=1)
randomgamma = RandomGamma((80,120),eps=None,always_apply=False,p=1)
RC = RandomContrast((0.2,0.8),False,1)

retinex = Retinex(model='MSR', sigma=[30,150,300],
       restore_factor=2.0, color_gain=6.0, gain=128.0, offset=128.0)

img_CLAHE = clahe(image=img)['image']

img_RG = randomgamma(image=img)['image']

img_RC = RC(image=img)['image']
img_retinex = retinex(img)

img_retinex_rs = (img_retinex * 255).astype(np.uint8)

images=[img,img_CLAHE,img_retinex_rs,tophat,img_RG,img_RC]
# titles=["Raw","CLAHE","Retinex","Tophat","Gamma","Random-Contrast"]
for i in range(6):
    plt.subplot(1,6,i+1),plt.imshow(images[i],"gray")
    # plt.title(titles[i])
    plt.xticks([])
    plt.yticks([])
plt.show()

plt.savefig("augment_list.png", dpi=400)#指定分辨率

### nertwork visualize
### tensorboardx

In [3]:
from mmdet.apis.inference import *
from mmdet.models import build_detector
import torch.nn as nn
import mmcv
from grad_cam import GradCAM
### 搭建模型
config = "/home/dlsuncheng/Projects/FSOD/FsMMdet/Model/few_shot/fc_cos-ft/frcn_cos_unfreeze_novel_30shot.py"
input_img = "/home/dlsuncheng/Projects/FSOD/FsMMdet/Datasets/NEU_DET/JPEGImages/crazing_1.jpg"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.config = mmcv.Config.fromfile(config)
        self.model= init_detector(self.config,device=device)

    def forward(self,x):
        result = self.model.forward_dummy(x)
        # result = self.model(x)
        return result
# input = torch.zeros([1 ,3, 200, 200])
image = mmcv.imread(input_img)
# height,width = image.shape[:2]
# input = {"image": image, "height": height, "width": width}

# input = input.to(device)
# input.to(device)

layer_name = "model.rpn_head.rpn_conv"

MyConvNet = ConvNet()
# grad_cam =  GradCAM(MyConvNet,layer_name)
# mask, box, class_id = grad_cam(input)  # cam mask
# grad_cam.remove_handlers()
### Grad_CAM(/++)

### 

/home/dlsuncheng/Packages/MMdet/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


In [8]:
import torchvision
import torch
# import hiddenlayer as h
import netron
from mmdet.apis.inference import *
from mmdet.models import build_detector
import torch.nn as nn
import mmcv
from torchviz import make_dot
# from tensorboardX import SummaryWriter
# writer = SummaryWriter('log')

config = "/home/dlsuncheng/Projects/FSOD/FsMMdet/Model/few_shot/fc_cos-ft/frcn_cos_unfreeze_novel_30shot.py"

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.config = mmcv.Config.fromfile(config)
        self.model= init_detector(self.config)

    def forward(self,x):
        result = self.model.forward_dummy(x)
        return result
    
input = torch.zeros([1 ,3, 200, 200])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input = input.to(device)
input.to(device)
MyConvNet = ConvNet()
MyConvNet.model.to(device)
y_rpn,y = MyConvNet(input)


# dot = make_dot(y, params=dict(MyConvNet.model.named_parameters()))
# dot.format = 'png'
# dot.render('FRCNN')

# writer.add_graph(MyConvNet, (input,))
# writer.close()

/home/dlsuncheng/Packages/MMdet/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


TypeError: forward() takes 2 positional arguments but 3 were given

### Few-shot dataset loader check